In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

import shared_utils
import pandas as pd
import geopandas as gpd

import gcsfs
from calitp_data_analysis import get_fs
from calitp_data_analysis import geography_utils, utils
fs = get_fs()
import re
import google.auth
import os
import gcsfs
credentials, project = google.auth.default()
fs = gcsfs.GCSFileSystem()

/opt/conda/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [10]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/ahsc_grant/'

In [11]:
acs_ca = read_parquet_from_gcs("tracts_ca_acs.parquet")

In [12]:
len(acs_ca)

9109

In [13]:
# Load job density data from GCS and select required columns
jobdata = pd.read_parquet(f"{GCS_FILE_PATH}job_density_2022.parquet")[['GEOID', 'jobs_tot']]

ArrowInvalid: google::cloud::Status(INVALID_ARGUMENT: Permanent error, with a last message of Could not create a OAuth2 access token to authenticate the request. The request was not sent, as such an access token is required to complete the request successfully. Learn more about Google Cloud authentication at https://cloud.google.com/docs/authentication. The underlying error message was: Unsupported credential type (external_account_authorized_user) when reading Application Default Credentials file from /home/jovyan/.config/gcloud/application_default_credentials.json. error_info={reason=, domain=, metadata={gcloud-cpp.retry.function=GetObjectMetadata, gcloud-cpp.retry.reason=permanent-error, gcloud-cpp.retry.original-message=Could not create a OAuth2 access token to authenticate the request. The request was not sent, as such an access token is required to complete the request successfully. Learn more about Google Cloud authentication at https://cloud.google.com/docs/authentication. The 